# **Importar datos a PostGIS**

## **1. Crear BD Espacial**

Crearemos la BD `geocoding`. Usaremos la opción `-T` para crear la base de datos a partir de la plantilla **`postgis`**, ademas indicaremos la opción `-w` para no solicitar la contraseña:

In [1]:
!createdb -U clopez -w geocoding -T postgis

Verificar que se creo la base de datos

In [3]:
!psql -U clopez -l | grep -i 'geocoding'

 geocoding | clopez   | UTF8         | libc                | es_ES.UTF-8 | es_ES.UTF-8 |                        |             | 


Ahora, vamos a dar persmisos en el esquema postgis y agregar al buscador de rutas el nombre del esquema **postgis**

In [4]:
!psql -U clopez -d geocoding -c "GRANT USAGE ON SCHEMA postgis TO public"

GRANT


In [5]:
!psql -d geocoding \
 -c "ALTER DATABASE geocoding \
     SET search_path=public,postgis,contrib"

ALTER DATABASE


## **2. Importar datos**

Primero crearemos un esquema donde importaremos la información:

In [6]:
!psql -U clopez -d geocoding \
 -c "CREATE SCHEMA cartografia"

CREATE SCHEMA


**Explorar los datos**

In [6]:
!ogrinfo /home/clopez/Documentos/data/Gas/gas.gpkg -al -so -nomd | grep -E 'Layer name|Feature Count'

Layer name: puerta
Feature Count: 2458235
Layer name: clientes_dep
Feature Count: 1067373
Layer name: barrios
Feature Count: 11528
Layer name: manzanas_dep
Feature Count: 143418
Layer name: ejes_viales
Feature Count: 386542


### **2.1 Importar clientes**

In [13]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -lco GEOMETRY_NAME=geom \
 -lco SCHEMA=cartografia \
 -lco SPATIAL_INDEX=GIST \
 -nln clientes \
 -nlt POINT \
 /home/clopez/Documentos/data/Gas/gas.gpkg clientes_dep

### **2.2. Importar barrios**

In [3]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -lco GEOMETRY_NAME=geom \
 -lco SCHEMA=cartografia \
 -lco SPATIAL_INDEX=GIST \
 -nln barrios \
 -nlt POLYGON \
 /home/clopez/Documentos/data/Gas/gas.gpkg barrios

### **2.3. Importar manzanas**

In [15]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -lco GEOMETRY_NAME=geom \
 -lco SCHEMA=cartografia \
 -lco SPATIAL_INDEX=GIST \
 -nln manzanas \
 -nlt POLYGON \
 /home/clopez/Documentos/data/Gas/gas.gpkg manzanas_dep

### **2.4. Importar calles**

In [17]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -lco GEOMETRY_NAME=geom \
 -lco SCHEMA=cartografia \
 -lco SPATIAL_INDEX=GIST \
 -nln ejes_viales \
 -nlt MULTILINESTRING \
 /home/clopez/Documentos/data/Gas/gas.gpkg ejes_viales

### **2.5. Importar puertas**

In [18]:
!ogrinfo /home/clopez/Documentos/data/Gas/gas.gpkg puerta -al -so -nomd

INFO: Open of `/home/clopez/Documentos/data/Gas/gas.gpkg'
      using driver `GPKG' successful.

Layer name: puerta
Geometry: Point
Feature Count: 2458235
Extent: (190369.260000, 8550236.785300) - (354418.824500, 8819279.905800)
Layer SRS WKT:
PROJCRS["WGS 84 / UTM zone 18S",
    BASEGEOGCRS["WGS 84",
        ENSEMBLE["World Geodetic System 1984 ensemble",
            MEMBER["World Geodetic System 1984 (Transit)"],
            MEMBER["World Geodetic System 1984 (G730)"],
            MEMBER["World Geodetic System 1984 (G873)"],
            MEMBER["World Geodetic System 1984 (G1150)"],
            MEMBER["World Geodetic System 1984 (G1674)"],
            MEMBER["World Geodetic System 1984 (G1762)"],
            MEMBER["World Geodetic System 1984 (G2139)"],
            MEMBER["World Geodetic System 1984 (G2296)"],
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]],
            ENSEMBLEACCURACY[2.0]],
        PRIMEM["Greenwich",0,
            ANGLE

In [20]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:32718 -t_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -lco GEOMETRY_NAME=geom \
 -lco SCHEMA=cartografia \
 -lco SPATIAL_INDEX=GIST \
 -nln apts \
 -nlt POINT \
 -skipfailures \
 /home/clopez/Documentos/data/Gas/gas.gpkg \
 -sql "SELECT OBJECTID as objectid, CODIGOPREDIO as cod_predio, \
              NUMEROLOTE as lote, NUMEROPUERTA as puerta, \
              CODIGODISTRITO as ubigeo, CODIGOSEGMENTOVIA as cod_via, \
              CODIGOMANZANA as cod_mzna, CODIGOPUERTA as cod_puerta, \
              FECHACREACION as created_date, FECHAMODIFICACION as last_edited_date,\
              geom \
        FROM puerta"

ERROR 1: Failed to reproject feature 1610976 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149068 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149157 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149188 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149190 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149208 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149232 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149234 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2149246 (geometry probably out of source or destination SRS).
ERROR 1: Failed to reproject feature 2150811 (geometry probably out of source or destination SRS).
ERROR 1: F

### **2.6. Importar distritos**

In [1]:
!ogrinfo /home/clopez/Documentos/data/inei/censal/censo.gpkg distritos -al -so -nomd

INFO: Open of `/home/clopez/Documentos/data/inei/censal/censo.gpkg'
      using driver `GPKG' successful.

Layer name: distritos
Geometry: Multi Polygon
Feature Count: 1876
Extent: (-81.328232, -18.350929) - (-68.652267, -0.038606)
Layer SRS WKT:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        MEMBER["World Geodetic System 1984 (G2296)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (La

In [2]:
!ogr2ogr -f PostgreSQL -a_srs EPSG:4326 \
 PG:"dbname=geocoding user=clopez" \
 -nln distritos \
 -nlt MULTIPOLYGON \
 -lco SCHEMA=cartografia \
 -lco GEOMETRY_NAME=geom \
 -lco SPATIAL_INDEX=GIST \
 /home/clopez/Documentos/data/inei/censal/censo.gpkg \
 -sql "SELECT FID as fid, UBIGEO as ubigeo, NOMBDIST as distrito, \
              NOMBPROV as provincia, NOMBDEP as departamento, \
              CAPITAL as capital, CCPP as ccpp, geom \
        FROM distritos"

validar cantidades

In [2]:
!psql -U clopez -d geocoding \
 -c "SELECT 'clientes' as tabla, count(*) as cantidad from cartografia.clientes \
     UNION \
     SELECT 'barrios' as tabla, count(*) as cantidad from cartografia.barrios \
     UNION \
     SELECT 'manzanas' as tabla, count(*) as cantidad from cartografia.manzanas \
     UNION \
     SELECT 'ejes_viales' as tabla, count(*) as cantidad from cartografia.ejes_viales \
     UNION \
     SELECT 'apt' as tabla, count(*) as cantidad from cartografia.apts \
     UNION \
     SELECT 'distritos' as tabla, count(*) as cantidad from cartografia.distritos"

    tabla    | cantidad 
-------------+----------
 apt         |  2458235
 barrios     |    11528
 clientes    |  1067373
 distritos   |     1874
 ejes_viales |   386542
 manzanas    |   143418
(6 filas)

